In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from Levenshtein import distance as levenshtein_distance
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.multiclass import OneVsRestClassifier

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ggentil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ggentil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ggentil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ggentil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
def pre_processamento_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"[^\w\s]", "", texto)
    texto = re.sub(r"\s+", " ", texto)
    stop_words = set(stopwords.words('english'))
    texto = " ".join([lemmatizer.lemmatize(word) for word in texto.split() if word not in stop_words])
    return texto

In [4]:
def calcular_similaridade_cosine(indice_filme, matriz_tfidf):
    return cosine_similarity(matriz_tfidf[indice_filme], matriz_tfidf).flatten()

def jaccard_similaridade(indice_filme, df_rec):
    conjunto1, conjunto2 = set(indice_filme.split()), set(df_rec.split())
    return len(conjunto1.intersection(conjunto2)) / len(conjunto1.union(conjunto2))

def calcular_similaridade_jaccard(indice_filme, df_rec):
    texto_filme = df_rec['colunas_combinadas_pre'].iloc[indice_filme]
    return np.array([jaccard_similaridade(texto_filme, texto) for texto in df_rec['colunas_combinadas_pre']])

def calcular_similaridade_levenshtein(indice_filme, df_rec):
    texto_filme = df_rec['colunas_combinadas_pre'].iloc[indice_filme]
    return np.array([1 / (1 + levenshtein_distance(texto_filme, texto)) for texto in df_rec['colunas_combinadas_pre']])


In [5]:
def calcular_similaridades(indice_filme, matriz_tfidf, df_rec):
    similaridades_cosine = calcular_similaridade_cosine(indice_filme, matriz_tfidf)
    similaridades_jaccard = calcular_similaridade_jaccard(indice_filme, df_rec)
    similaridades_levenshtein = calcular_similaridade_levenshtein(indice_filme, df_rec)

    similaridades_cosine = (similaridades_cosine - np.min(similaridades_cosine)) / (np.max(similaridades_cosine) - np.min(similaridades_cosine))
    similaridades_jaccard = (similaridades_jaccard - np.min(similaridades_jaccard)) / (np.max(similaridades_jaccard) - np.min(similaridades_jaccard))
    similaridades_levenshtein = (similaridades_levenshtein - np.min(similaridades_levenshtein)) / (np.max(similaridades_levenshtein) - np.min(similaridades_levenshtein))
    
    similaridades_agregadas = similaridades_cosine + similaridades_jaccard + similaridades_levenshtein
    return similaridades_agregadas

In [6]:
def recomendar_filme(filme_assistido, df_rec, matriz_tfidf, n_recomendacoes):
    try:
        if filme_assistido.lower() not in df_rec['Series_Title'].str.lower().values:
            return print(f"Filme '{filme_assistido}' não encontrado no dataset.")

        indice_filme = df_rec[df_rec['Series_Title'].str.lower() == filme_assistido.lower()].index[0]
        similaridades_agregadas = calcular_similaridades(indice_filme, matriz_tfidf, df_rec)

        indices_similares = similaridades_agregadas.argsort()[-(n_recomendacoes+1):][::-1]
        indices_similares = indices_similares[indices_similares != indice_filme]
        filmes_recomendados = df_rec['Series_Title'].iloc[indices_similares].tolist()
        return filmes_recomendados

    except Exception as e:
        return print(f"Ocorreu um erro: {e}")

In [7]:
def preparar_dataset():
    # Carregar o dataset de filmes
    df_rec = pd.read_csv("dados_tratados.csv")

    # Aplicar pré-processamento na coluna "colunas_combinadas"
    df_rec['colunas_combinadas'] = df_rec.apply(lambda x: f"{x['Series_Title']} {x['Certificate']} {x['Genre']} {x['Overview']} {x['Director']}", axis=1)
    df_rec['colunas_combinadas_pre'] = df_rec['colunas_combinadas'].apply(pre_processamento_texto)

    # Vetorizar TF-IDF para "colunas_combinadas_pre"
    vetor_tfidf = TfidfVectorizer()
    matriz_tfidf = vetor_tfidf.fit_transform(df_rec['colunas_combinadas_pre'])

    # Processamento dos gêneros usando MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    generos_codificados = mlb.fit_transform(df_rec['Genre'].apply(lambda x: x.split(", ")))

    # Concatenar matriz TF-IDF com vetores de gênero usando hstack
    matriz_tfidf_com_generos = hstack([matriz_tfidf, generos_codificados.astype(float)])

    # Aplicar Chi-Quadrado
    seletor = SelectKBest(chi2, k=1000)
    matriz_tfidf_com_generos_reduzida = seletor.fit_transform(matriz_tfidf_com_generos, generos_codificados)
    return df_rec, matriz_tfidf_com_generos, matriz_tfidf_com_generos_reduzida, generos_codificados


In [8]:
def main():
    df_rec, matriz_tfidf_com_generos, matriz_tfidf_com_generos_reduzida, generos_codificados = preparar_dataset()
    
    # Solicitar ao usuário o nome do filme assistido
    filme_assistido = input("Digite o nome do filme assistido: ").strip()

    # Validar entrada do usuário
    if not filme_assistido:
        print("Por favor, insira um título de filme válido.")
    else:
        # Obter recomendações usando a matriz TF-IDF com gêneros reduzida
        filmes_recomendados = recomendar_filme(filme_assistido, df_rec, matriz_tfidf_com_generos_reduzida, n_recomendacoes=8)

        # Imprimir recomendações se houver
        if filmes_recomendados:
            print(f"Filmes Recomendados para '{filme_assistido}':")
            for filme in filmes_recomendados:
                print(f"- {filme}")

            # Dividir dados para treinamento e teste
            X_treino, X_teste, y_treino, y_teste = train_test_split(matriz_tfidf_com_generos, generos_codificados, test_size=0.2, random_state=42)

            # Configurar e treinar SVM com Grid Search para otimização de parâmetros
            param_grid = {'estimator__C': [0.1, 1, 10], 'estimator__kernel': ['linear', 'rbf']}
            grid_search = GridSearchCV(OneVsRestClassifier(SVC()), param_grid, cv=5)
            grid_search.fit(X_treino, y_treino)

            # Usar cross_val_score para avaliar o modelo
            scores = cross_val_score(grid_search.best_estimator_, matriz_tfidf_com_generos, generos_codificados, cv=5, scoring='accuracy')
            print("_________________________________________________________________")
            print(f"Validação Cruzada Acurácia: {np.mean(scores)}")

            # Avaliar o modelo nos dados de teste
            y_pred = grid_search.predict(X_teste)
            print(f"Acurácia do SVM: {accuracy_score(y_teste, y_pred)}")
            print(f"Melhores parâmetros: {grid_search.best_params_}")
            print(f"Precisão: {precision_score(y_teste, y_pred, average='weighted')}")
            print(f"Recall: {recall_score(y_teste, y_pred, average='weighted')}")
            print(f"F1-score: {f1_score(y_teste, y_pred, average='weighted')}")

            # Relatório de classificação
            print("\nRelatório de Classificação:")
            print(classification_report(y_teste, y_pred))


In [9]:
if __name__ == "__main__":
    main()


Filmes Recomendados para 'The Godfather':
- The Godfather: Part III
- The Godfather: Part II
- Un prophète
- Casino
- Scarface
- Once Were Warriors
- Drive
- La haine
_________________________________________________________________
Validação Cruzada Acurácia: 1.0
Acurácia do SVM: 1.0
Melhores parâmetros: {'estimator__C': 1, 'estimator__kernel': 'linear'}
Precisão: 1.0
Recall: 1.0
F1-score: 1.0

Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00        48
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        27
           4       1.00      1.00      1.00        42
           5       1.00      1.00      1.00        31
           6       1.00      1.00      1.00       145
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00         9
           9       1.00      1.0